In [1]:
from reggie.ingestion.utils import format_column_name
from reggie.configs.configs import Config
import logging
import pandas as pd
import datetime
from io import StringIO
from datetime import datetime
import time
import numpy as np
from functools import reduce
from collections import defaultdict, OrderedDict
from dateutil import parser
import json
%load_ext line_profiler

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
voter_file = '/home/tommi/Downloads/2021-02-22/Statewide/ADAMS FVE 20210222.txt'
zone_codes_file = '/home/tommi/Downloads/2021-02-22/Statewide/ADAMS Zone Codes 20210222.txt'
zone_types_file = '/home/tommi/Downloads/2021-02-22/Statewide/ADAMS Zone Types 20210222.txt'
election_map = '/home/tommi/Downloads/2021-02-22/Statewide/ADAMS Election Map 20210222.txt'
# voter_file = '/home/tommi/Downloads/2021-02-22/Statewide/AlFVESm.txt'
# voter_file = '/home/tommi/Downloads/2021-02-22/Statewide/ALLEGHENY FVE 20210222.txt'
# zone_codes_file = '/home/tommi/Downloads/2021-02-22/Statewide/ALLEGHENY Zone Codes 20210222.txt'
# zone_types_file = '/home/tommi/Downloads/2021-02-22/Statewide/ALLEGHENY Zone Types 20210222.txt'
# election_map = '/home/tommi/Downloads/2021-02-22/Statewide/ALLEGHENY Election Map 20210222.txt'
# voter_file = '/home/tommi/Downloads/2021-02-22/Statewide/BEDFORD FVE 20210222.txt'
# zone_codes_file = '/home/tommi/Downloads/2021-02-22/Statewide/BEDFORD Zone Codes 20210222.txt'
# zone_types_file = '/home/tommi/Downloads/2021-02-22/Statewide/BEDFORD Zone Types 20210222.txt'
# election_map = '/home/tommi/Downloads/2021-02-22/Statewide/BEDFORD Election Map 20210222.txt'
# pd.set_option('display.expand_frame_repr', False)

In [3]:
config = Config(state='pennsylvania')
dfcols = config["ordered_columns"][:-3]
df_cols = config["ordered_columns"]
elections = 40
# quick_d = {}
# for i in range(elections):
#     dfcols.extend(["district_{}".format(i + 1)])
#     district_col_lst.append("district_{}".format(i + 1))
# for i in range(elections):
#     dfcols.extend(["election_{}_vote_method".format(i + 1)])
#     quick_d["election_{}_vote_method".format(i + 1)] = str
#     dfcols.extend(["election_{}_party".format(i + 1)])
#     quick_d["election_{}_party".format(i + 1)] = str
dfcols.extend(config["ordered_columns"][-3:])
# dfcols
''' Low Priority change to remove these small loops'''
## different way of making columns, indexing from 0 now low prio
# districts = []
# vote_data = []
# election_party = []
# for i in range(1, elections+1):
#     districts.append(f'district_{i}')
#     vote_data.extend([f'election_{i}_vote_method',f'election_{i}_party'])

# df_cols.extend(districts)
# df_cols.extend(vote_data)
# df_cols.extend(config["ordered_columns"][-3:])
## print(df_cols == dfcols)
dtypes = {row: 'str' for row in config['ordered_columns']}


In [4]:
# vdf = pd.read_csv(voter_file, sep="\t", names=dfcols, dtype=config['dtypes'])
vdf = pd.read_csv(voter_file, sep="\t", names=dfcols, dtype=dtypes)
# edf = pd.read_csv(election_map, sep="\t", names=["county", "number", "title", "date"])
edf = pd.read_csv(election_map, sep="\t", names=["county", "number", "title", "date"], dtype={'county': str, 'number': str, 'title': str, 'date': str} )

zdf = pd.read_csv(zone_codes_file, sep="\t", names=["county", "number", "code", "title"])
tdf = pd.read_csv(zone_types_file, sep="\t", names=["county", "number", "abbr", "title"])
## new titles because the names were bad
tzdf = pd.read_csv(zone_codes_file, sep="\t", names=["county_name", "zone_number", "zone_code", "zone_description"])
ttdf = pd.read_csv(zone_types_file, sep="\t", names=["county_name", "zone_number", "zone_short_name", "zone_long_name"])

In [5]:
# vdf
# dtypes

In [6]:
vdf = vdf.replace('"')
edf = edf.replace('"')
zdf = zdf.replace('"')
edf.set_index = edf["number"]

pd.options.display.max_colwidth = 100
districtsl = vdf.iloc[:, 30:70]
vote_historyl = vdf.iloc[:, 70:150]


In [7]:
# vote_columns = vote_historyl.columns.to_list()
# district_columns = districtsl.columns.to_list()
pd.set_option('display.max_columns', None)
vdf

,voter_id,title,surname,first_name,middle_name,suffix,gender,dob,registration_date,voter_status,status_change_date,party_identifier,house_number,house_number_suffix,street_name,apartment_number,address_line_2,city,state,zip,mail_address_1,mail_address_2,mail_city,mail_state,mail_zip,last_vote_date,precinct_code,precinct_split_id,date_last_changed,legacy_id,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,district_11,district_12,district_13,district_14,district_15,district_16,district_17,district_18,district_19,district_20,district_21,district_22,district_23,district_24,district_25,district_26,district_27,district_28,district_29,district_30,district_31,district_32,district_33,district_34,district_35,district_36,district_37,district_38,district_39,district_40,election_1_vote_method,election_1_party,election_2_vote_method,election_2_party,election_3_vote_method,election_3_party,election_4_vote_method,election_4_party,election_5_vote_method,election_5_party,election_6_vote_method,election_6_party,election_7_vote_method,election_7_party,election_8_vote_method,election_8_party,election_9_vote_method,election_9_party,election_10_vote_method,election_10_party,election_11_vote_method,election_11_party,election_12_vote_method,election_12_party,election_13_vote_method,election_13_party,election_14_vote_method,election_14_party,election_15_vote_method,election_15_party,election_16_vote_method,election_16_party,election_17_vote_method,election_17_party,election_18_vote_method,election_18_party,election_19_vote_method,election_19_party,election_20_vote_method,election_20_party,election_21_vote_method,election_21_party,election_22_vote_method,election_22_party,election_23_vote_method,election_23_party,election_24_vote_method,election_24_party,election_25_vote_method,election_25_party,election_26_vote_method,election_26_party,election_27_vote_method,election_27_party,election_28_vote_method,election_28_party,election_29_vote_method,election_29_party,election_30_vote_method,election_30_party,election_31_vote_method,election_31_party,election_32_vote_method,election_32_party,election_33_vote_method,election_33_party,election_34_vote_method,election_34_party,election_35_vote_method,election_35_party,election_36_vote_method,election_36_party,election_37_vote_method,election_37_party,election_38_vote_method,election_38_party,election_39_vote_method,election_39_party,election_40_vote_method,election_40_party,home_phone,county,mail_country
0,001003958-01,NaN,WEIKERT,MARK,K,NaN,M,11/20/1932,01/01/1955,A,01/11/2017,R,13,NaN,FULTON DR,NaN,NaN,NEW OXFORD,PA,17350,NaN,NaN,NaN,NaN,NaN,11/03/2020,44,44-1,10/17/2020,NaN,44,NaN,SC6,TS17,MD03,STH193,STS33,USC13,CO01,NaN,TS17,NaN,44-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AP,R,NaN,NaN,AP,R,NaN,NaN,AP,R,AP,R,AP,R,MB,R,MB,R,NaN,NaN,NaN,NaN,NaN,NaN,AP,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADAMS,NaN
1,001006201-01,NaN,STOUFFER,JOHN,C,NaN,M,02/03/1927,01/01/1959,A,09/06/2017,D,6375,NaN,CHAMBERSBURG RD,#105,NaN,FAYETTEVILLE,PA,17222,NaN,NaN,NaN,NaN,NaN,11/03/2020,18,18-1,10/09/2020,NaN,18,NaN,SC8,TS5,MD04,STH091,STS33,USC13,CO01,NaN,TS5,NaN,18-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AP,D,AP,D,NaN,NaN,NaN,NaN,MB,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7173522009,ADAMS,NaN
2,001006466-01,NaN,ADKINS,ELLEN,E,NaN,F,09/06/1920,01/01/1960,I,07/10/2018,R,2990,NaN,CARLISLE PIKE,NaN,NaN,NEW OXFORD,PA,17350,NaN

In [8]:
edf

,county,number,title,date
0,ADAMS,1,2005 MUNICIPAL ELECTION,11/08/2005
1,ADAMS,2,2002 GENERAL ELECTION,11/05/2002
2,ADAMS,3,2014 GENERAL ELECTION,11/04/2014
3,ADAMS,4,2015 MUNICIPAL PRIMARY,05/19/2015
4,ADAMS,5,2015 MUNICIPAL ELECTION,11/03/2015
5,ADAMS,6,2016 GENERAL PRIMARY,04/26/2016
6,ADAMS,7,2016 GENERAL ELECTION,11/08/2016
7,ADAMS,8,2017 MUNICIPAL PRIMARY,05/16/2017
8,ADAMS,9,2013 MUNICIPAL ELECTION,11/05/2013
9,ADAMS,10,2018 GENERAL PRIMARY,05/15/2018


In [9]:
sorted_codes = []
counts = pd.Series()
# sorted_code_dict = defaultdict(int)
sorted_code_dict = defaultdict(defaultdict)
code_dict = {}
edf["election_list"] = edf["title"] + " " + edf["date"]
edf["sorted_codes_col"] = edf["date"] + "_" + edf["title"]
edf["sorted_codes_col"] = edf["sorted_codes_col"].str.replace(" ", "_")
vote_columns = vdf.columns[70:150].to_list()
district_columns = vdf.columns[30:70].to_list()

def final_format(df, edf, zdf, tdf):
    def list_map(df_sub, columns, zone_dict=None):
        def mapping(li, zone_dict=zone_dict):
            if zone_dict is None:
                li = [x for x in li if x != "nan"]
                return li
            else:
                li = [
                    zone_dict[x]
                    for x in li
                    if x != "nan" and x in zone_dict
                ]
                return li

        return pd.Series(
            map(mapping, df_sub[columns].values.astype(str).tolist())
        )
    # create a dict of the formatted string election data with the index number in the given file, this
    # corresponds to the column index beginning at the start of the vote columns in the dataframe
    # Index begins starting at 1
    # Todo: This upper is causing a pretty big diff but I am not sure if I care?
    election_map = pd.Series(
        edf.election_list.values, index=edf.number
    ).to_dict()

    # merge the zone files together to consolidate the information in one dataframe
    # todo: just add the info to the one dataframe rather than create a new one
    zdf = zdf.merge(tdf, how="left", on="zone_number")
    # format a column field that contains the zone description and the name so
    # that it matches the current district field
    zdf["combined"] = (
        zdf["zone_description"]
        + " Type: "
        + zdf["zone_long_name"]
    )

    # create a dict that utilizes the zone code as the key and the long name string as the value
    zone_dict = dict(
        zip(zdf.zone_code.astype(str), zdf.combined)
    )

    # Gather the pairs of election columns to iterate over both at the same time to collect the information
    # contained in both of the columns per election
    vote_column_list = list(
        zip(df.columns[70:150:2], df.columns[71:150:2])
    )

    # todo: probably don't need this dataframe either
    # district_df = df[district_columns]

    # get the value from the eleciton map key, then combine it with the value in the party and vote type cells
    # Create a history dataframe containing, as cells the election name as gathered in the election file, the
    # vote type (AP, A etc), and the party all separated by spaces
    # The columns are all named election_#_vote_type but the cells contain the relevant information
    vote_hist_df = pd.DataFrame(
        {
            i: election_map[i.split("_")[1]]
            + " "
            + df[i]
            + " "
            + df[j]
            for i, j in vote_column_list
            if i.split("_")[1] in election_map
        }
    )

    # counts for the metadata
    counts = vote_hist_df.count()
    for i in counts.index:
        current_key = election_map[i.split("_")[1]]

        # Metadata needs to be _ separated not space
        current_key = "_".join(current_key.split())
        if current_key in sorted_code_dict:
            sorted_code_dict[current_key]["count"] += int(counts[i])
        else:
            # todo: consolidate this into one dict call if possible
            current_date = edf.loc[edf["number"] == i.split("_")[1]][
                "date"
            ].values[0]
            new_dict_entry = defaultdict(str)
            new_dict_entry["date"] = current_date
            new_dict_entry["count"] = int(counts[i])
            sorted_code_dict[current_key] = new_dict_entry
    # converts the dataframe to a series that contains the list of elections participate in indexed on position
    vote_hist_df = list_map(vote_hist_df, vote_hist_df.columns)
    districts = list_map(df[district_columns], district_columns, zone_dict)

    df["all_history"] = vote_hist_df
    df["districts"] = districts
    df.drop(vote_columns, axis=1, inplace=True)
    df.drop(district_columns, axis=1, inplace=True)

    cols_to_check = [
        col
        for col in list(df.columns)
        if col not in vote_columns and col not in district_columns
    ]
    return df


<ipython-input-9-537d48904151>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  counts = pd.Series()
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [10]:
%prun tempdfinal = final_format(vdf, edf, tzdf, ttdf)


In [ ]:
sorted_codes = []
counts = pd.Series()
# sorted_code_dict = defaultdict(int)
sorted_code_dict = defaultdict(defaultdict)
code_dict = {}
edf["election_list"] = edf["title"] + " " + edf["date"]
edf["sorted_codes_col"] = edf["date"] + "_" + edf["title"]
edf["sorted_codes_col"] = edf["sorted_codes_col"].str.replace(" ", "_")
vote_columns = vdf.columns[70:150].to_list()
print(vote_columns)
district_columns = vdf.columns[30:70].to_list()
def new_format(df, edf, zdf, tdf):
    zdf.drop_duplicates("zone_code", inplace=True)
    zdf.drop_duplicates("zone_description", inplace=True)
    

    def list_map(df, columns, zone_dict=None):
        def mapping(li, zone_dict=zone_dict):
            if zone_dict == None:
                li = [x for x in li if x != 'nan']
                return li
            else:
                li = [zone_dict[x] for x in li if x!= 'nan' and x in zone_dict]
                return li
        return pd.Series(map(mapping, df[columns].values.astype(str).tolist()))


    #gather the election names
    election_map = pd.Series(edf.election_list.str.upper().values,index=edf.number).to_dict()
    # merge the zone files together

#     print(*vote_columns, sep='\n')
#     print(*tdf["zone_number"].unique(), sep='\n')
    unholy_union = zdf.merge(tdf, how='left', on='zone_number')
    unholy_union["combined"] = unholy_union["zone_description"] + " Type: " + unholy_union["zone_long_name"] 
    zone_dict = dict(zip(unholy_union.zone_code.astype(str), unholy_union.combined))
    
    # gathers the pairs of election columns
    vote_li = list(zip(df.columns[70:150:2],df.columns[71:150:2]))
    new_df = df[district_columns]

# WHAT IS THIS?
    # get the value from the eleciton map key, then combine it with the value in the party and vote type cells
    # annoying error for a county in pandas where eval fails with a string type in this dataframe?
    temp = pd.DataFrame({i :election_map[i.split("_")[1]] + " " + df[i] + " " + df[j] for i,j in vote_li})
    # ~~~~~~~~~~~~~sorted codes meta stuff here
    # Enumerate over sorted codes and do the index that way, ez pz
    # Okay so this is irritating but form this elsewhere
#     sorted_codes.extend(edf['sorted_codes_col'].str.replace(" ", "_").values)
    print("temp", temp)
    counts = temp.count()
#     print(counts.values, counts.index)
    for i in counts.index:
#         print(i)
#         meta_dict = {'date':}
        current_key = election_map[i.split("_")[1]]
        current_key = "_".join(current_key.split())
        if current_key in sorted_code_dict:
            # needs to be an int to jsonify, json apparently hates numpy int64s
            sorted_code_dict[current_key]['count'] += int(counts[i])
        else:
            current_date = edf.loc[edf['number'] == i.split("_")[1]]['date'].values[0]
            sorted_code_dict[current_key] = {'date': current_date, 'count': int(counts[i])}

#     print(counts[0])
#     print(counts, type(counts))
    # ~~~~~~~~~~~end sorted codes
    # Unnecessary
    cols_to_drop = temp.columns

    temp_ser = list_map(temp, cols_to_drop)
    district_ser = list_map(new_df, district_columns, zone_dict)
#     print(district_ser)

    df['all_history'] = temp_ser
    df['districts'] = district_ser
#     df.drop(vote_columns, axis=1, inplace=True)
#     df.drop(district_columns, axis=1, inplace=True)

#     df["all_history"] = df[vote_columns].apply(return_election_string, args=(election_map,), axis=1)
#     df["districts"] = df[district_columns].apply(return_district_string, args=(zone_dict,), axis=1)
#     print(dict(sorted(sorted_code_dict.items(), key = lambda x: x[1]['date'])))
    sorted_keys = sorted(sorted_code_dict.items(), key=lambda x: parser.parse(x[1]['date']))
    combined_dict = {}
    for index, key in enumerate(sorted_keys):
        sorted_code_dict[key[0]]['index'] = str(index)
#         print(type(sorted_code_dict[key[0]]['date']), type(sorted_code_dict[key[0]]['index']), type(sorted_code_dict[key[0]]['count']))
        combined_dict[key[0]] = sorted_code_dict[key[0]]
#         print(key[0])
#         print(dict(sorted_code_dict[key[0]]))
        sorted_codes.append(key[0])
    sorted_code_dict1 = dict(sorted_code_dict)

    meta = {
    "message": "pennsylvania_{}".format(datetime.now().isoformat()),
    "array_encoding": json.dumps(sorted_code_dict),
    "array_decoding": json.dumps(sorted_codes),}
    return df

In [ ]:
# %prun tempdf2 = new_format(vdf, edf, tzdf, ttdf)
# %prun -T finalnew 
tempdf2 = new_format(vdf, edf, tzdf, ttdf)
# print(tempdf2['districts'])

In [ ]:
tempdf2

In [11]:
# print(*list(tempdf2.columns), sep='\n')
vdf = pd.read_csv(voter_file, sep="\t", names=dfcols)
vdf = vdf.replace('"')

/home/tommi/.cache/pypoetry/virtualenvs/inspector-nJVqbbXH-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (12,21,24,29,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:


def old_format(df, edf, zdf, tdf):
    for i in range(elections):
        s = pd.Series(index=df.index)
        # Blair isn't sending all their election codes
        try:
            s[:] = (
                edf.iloc[i]["title"] + " " + edf.iloc[i]["date"] + " "
            )
        except IndexError:
            s[:] = "UNSPECIFIED"
        df["election_{}".format(i)] = (
            s
            + df["election_{}_vote_method".format(i + 1)].apply(str)
            + " "
            + df["election_{}_party".format(i + 1)]
        )
        df.loc[
            df["election_{}_vote_method".format(i + 1)].isna(),
            "election_{}".format(i),
        ] = pd.np.nan
        df = df.drop("election_{}_vote_method".format(i + 1), axis=1)
        df = df.drop("election_{}_party".format(i + 1), axis=1)
        # just mapping to the zdf here
        df["district_{}".format(i + 1)] = df[
            "district_{}".format(i + 1)
        ].map(
            zdf.drop_duplicates("code")
            .reset_index()
            .set_index("code")["title"]
        )
        df["district_{}".format(i + 1)] += ", Type: " + df[
            "district_{}".format(i + 1)
        ].map(
            zdf.drop_duplicates("title")
            .reset_index()
            .set_index("title")["number"]
        ).map(
            tdf.set_index("number")["title"]
        )

    df["all_history"] = df[
        ["election_{}".format(i) for i in range(elections)]
    ].values.tolist()
    df["all_history"] = df["all_history"].map(
        lambda L: list(filter(pd.notna, L))
    )
    df["districts"] = df[
        ["district_{}".format(i + 1) for i in range(elections)]
    ].values.tolist()
    df["districts"] = df["districts"].map(
        lambda L: list(filter(pd.notna, L))
    )
    for i in range(elections):
        df = df.drop("election_{}".format(i), axis=1)
        df = df.drop("district_{}".format(i + 1), axis=1)
    return df

In [13]:
# %prun -s cumulative -q -l 10 -T prun1 
# %prun -T finalold  oldtempdf = old_format(vdf, edf, zdf, tdf)
%prun oldtempdf = old_format(vdf, edf, zdf, tdf)

<ipython-input-12-c7313f49ef37>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(index=df.index)
<ipython-input-12-c7313f49ef37>:20: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  ] = pd.np.nan


In [ ]:
procal = pd.read_csv('/home/tommi/Downloads/procal.csv', low_memory=False)

In [ ]:
# idlist = list(tempdf2['voter_id'].values)

# would be small if this was not the whole file in memory, use when using small file
# smallist = procal.loc[procal['voter_id'].isin(idlist)]
# print(smallist)
print(procal.shape)
print(tempdf2.shape)
print(procal.tail(5))
print(tempdf2.tail(5))
comp1 = procal[['voter_id', 'districts', 'all_history']]
comp2 = tempdf2[['voter_id', 'districts', 'all_history']]
complist = comp2['all_history']
# complist2 = comp1['all_history']
# complist = procal['all_history'].apply(str)
complist2 = tempdf2['all_history']
# print(complist)

# print(complist2)
# temp3 = [x for x in complist if x not in complist2]


if complist.equals(complist2):
    print("No worries, mate")
else:
    print("Some worries")
#     print(temp3)
